In [1]:
import pandas as pd
import numpy as np
import pacmap
from sklearn.decomposition import PCA
from pyod.models.iforest import IForest
from sklearn.model_selection import train_test_split
import plotly.express as px

In [2]:
df = pd.read_csv("fft_dataset.csv", index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,...,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399
6_01_0,0.131477,0.040664,0.021781,0.016047,0.241540,0.049441,0.065172,0.349460,0.476330,0.386294,...,0.000092,0.000070,0.000042,0.000073,0.000092,0.000071,0.000042,0.000072,0.000092,0.000072
6_01_1,0.115790,0.033437,0.045090,0.037052,0.060328,0.065047,0.090258,0.186483,0.435931,0.238401,...,0.000018,0.000021,0.000046,0.000041,0.000009,0.000029,0.000047,0.000035,0.000002,0.000035
6_01_10,0.005463,0.078892,0.009493,0.136006,0.058788,0.058395,0.300883,0.224023,1.000000,0.677277,...,0.000072,0.000157,0.000122,0.000031,0.000136,0.000151,0.000053,0.000099,0.000160,0.000099
6_01_11,0.110545,0.009117,0.026015,0.037528,0.062674,0.072264,0.082788,0.136266,0.472449,0.432665,...,0.000222,0.000229,0.000231,0.000225,0.000222,0.000228,0.000232,0.000226,0.000221,0.000226
6_01_12,0.125418,0.031603,0.048844,0.026526,0.128466,0.074944,0.107386,0.383622,0.633578,0.299405,...,0.000052,0.000108,0.000110,0.000057,0.000027,0.000096,0.000116,0.000078,0.000008,0.000078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6_60_5,0.071865,0.004768,0.003300,0.011508,0.008962,0.010774,0.001901,0.100205,0.240953,0.056392,...,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005
6_60_6,0.298776,0.158232,0.082080,0.036483,0.101728,0.085802,0.072058,0.126860,0.250605,0.041331,...,0.000017,0.000016,0.000008,0.000012,0.000018,0.000015,0.000007,0.000014,0.000018,0.000014
6_60_7,0.178895,0.040466,0.052830,0.055122,0.031937,0.033674,0.073049,0.090692,0.503542,0.300539,...,0.000165,0.000141,0.000103,0.000065,0.000060,0.000096,0.000135,0.000162,0.000171,0.000162
6_60_8,0.074978,0.008684,0.010270,0.021518,0.033130,0.030203,0.027108,0.061079,0.156846,0.074700,...,0.000050,0.000034,0.000026,0.000044,0.000051,0.000038,0.000025,0.000041,0.000051,0.000041


In [3]:
# train-test split
X = df.to_numpy()
y = df.index.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=False)

In [4]:
# use iForest for anomaly detection
model = IForest(contamination=0.1, random_state=42)
model.fit(X_train)
# predict anomalies
y_train_pred = model.predict(X_train)
# get anomaly scores
y_train_scores = model.decision_scores_
# get anomaly scores for test set
y_test_scores = model.decision_function(X_test)
# get anomaly labels for test set
y_test_pred = model.predict(X_test)

In [5]:
all_scores = np.concatenate((y_train_scores, y_test_scores))
all_scores

array([-0.09231   , -0.0783146 , -0.08698021, ..., -0.06178318,
       -0.09948243, -0.08303663], shape=(3000,))

In [6]:
df_results = pd.DataFrame(index= df.index)
# add a column train or test to the original dataframe
df_results['set'] = ['train' if i in y_train else 'test' for i in y]
# add a column for anomaly scores to the original dataframe
df_results['anomaly_score'] = all_scores
df_results

,set,anomaly_score
6_01_0,train,-0.092310
6_01_1,train,-0.078315
6_01_10,train,-0.086980
6_01_11,train,-0.090074
6_01_12,train,-0.065836
...,...,...
6_60_5,test,-0.091232
6_60_6,test,0.114529
6_60_7,test,-0.061783
6_60_8,test,-0.099482


In [7]:
# check how many components to keep for 99% variance explained
pca = PCA(n_components=0.99)
pca.fit(X_train)
n_components = pca.n_components_
print(f"Number of components to keep for 99% variance explained: {n_components}")

Number of components to keep for 99% variance explained: 131


In [8]:
# Variance explained by the top 10 components
explained_variance = pca.explained_variance_ratio_
print(f"Explained variance by each component: {explained_variance[:10]}")

Explained variance by each component: [0.15486198 0.13040442 0.10576425 0.07933151 0.06271987 0.04625833
 0.03752897 0.03182597 0.02800717 0.02608121]


In [9]:
X_train_pca = pca.transform(X_train)

In [10]:
# calculate reconstruction error for PCAm (training set)
X_train_pca_inv = pca.inverse_transform(X_train_pca)
reconstruction_error_pca_train = ((X_train - X_train_pca_inv) ** 2).mean(axis=1) # MSE
print(f"Reconstruction error for PCA: {reconstruction_error_pca_train.mean()}")

Reconstruction error for PCA: 5.407241429923077e-05


In [11]:
# calculate reconstruction error for PCAm (test set)
X_test_pca = pca.transform(X_test)
X_test_pca_inv = pca.inverse_transform(X_test_pca)
reconstruction_error_pca_test = ((X_test - X_test_pca_inv) ** 2).mean(axis=1) # MSE
print(f"Reconstruction error for PCA on test set: {reconstruction_error_pca_test.mean()}")

Reconstruction error for PCA on test set: 5.5658096817994724e-05


In [12]:
all_recostruction = np.concatenate((X_train_pca_inv, X_test_pca_inv))
all_recostruction.shape

(3000, 3400)

In [13]:
all_absolute_reconstruction_error = np.abs(X - all_recostruction)
df_errors = pd.DataFrame(all_absolute_reconstruction_error, index=df.index, columns=df.columns)
df_errors

,0,1,2,3,4,5,6,7,8,9,...,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399
6_01_0,0.014790,0.001416,0.022860,0.033189,0.164342,0.022550,0.038045,0.004257,0.002465,0.001508,...,0.000046,6.672786e-05,0.000103,0.000063,0.000060,0.000074,0.000094,0.000071,0.000051,0.000071
6_01_1,0.004220,0.016162,0.004211,0.015267,0.015560,0.015245,0.004124,0.003060,0.003122,0.000962,...,0.000115,1.164994e-04,0.000088,0.000097,0.000133,0.000105,0.000084,0.000105,0.000133,0.000105
6_01_10,0.106817,0.015537,0.054044,0.056002,0.038953,0.098687,0.081234,0.032280,0.006890,0.004808,...,0.000037,3.519401e-05,0.000001,0.000086,0.000018,0.000031,0.000070,0.000028,0.000046,0.000028
6_01_11,0.014441,0.032865,0.017155,0.009737,0.005275,0.006152,0.005015,0.001475,0.000281,0.003417,...,0.000153,1.593950e-04,0.000172,0.000153,0.000154,0.000166,0.000161,0.000152,0.000165,0.000152
6_01_12,0.026249,0.023277,0.001921,0.030432,0.030695,0.021742,0.012976,0.002079,0.001276,0.000685,...,0.000042,8.370780e-07,0.000008,0.000047,0.000071,0.000006,0.000011,0.000026,0.000076,0.000026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6_60_5,0.029150,0.014397,0.001194,0.002358,0.008880,0.019109,0.009993,0.000400,0.001845,0.005014,...,0.000043,4.500048e-05,0.000041,0.000051,0.000028,0.000025,0.000038,0.000060,0.000038,0.000060
6_60_6,0.145400,0.120923,0.030300,0.005665,0.028767,0.002229,0.033187,0.015143,0.011163,0.006083,...,0.000087,1.202126e-04,0.000128,0.000112,0.000105,0.000115,0.000108,0.000126,0.000097,0.000126
6_60_7,0.050206,0.015064,0.010934,0.014968,0.006929,0.021962,0.021272,0.014128,0.007551,0.004632,...,0.000036,5.070246e-05,0.000098,0.000142,0.000132,0.000116,0.000065,0.000037,0.000020,0.000037
6_60_8,0.014142,0.008707,0.003656,0.007464,0.017497,0.003294,0.004378,0.007624,0.001937,0.000765,...,0.000008,3.167977e-05,0.000032,0.000018,0.000001,0.000008,0.000029,0.000033,0.000006,0.000033


In [14]:
all_reconstruction_error = ((X - all_recostruction) ** 2).mean(axis=1) # MSE
all_reconstruction_error.shape

(3000,)

In [15]:
# calculate for each test sample the reconstruction error
reconstruction_error_pca_df = pd.DataFrame({
    'Sample': y,
    'Reconstruction Error': all_reconstruction_error
})
reconstruction_error_pca_df.sort_values(by='Reconstruction Error', ascending=False).head(10)

,Sample,Reconstruction Error
2997,6_60_7,0.000560
2466,6_50_23,0.000557
2996,6_60_6,0.000502
2973,6_60_3,0.000469
2342,6_47_47,0.000424
2061,6_42_19,0.000416
775,6_16_31,0.000400
1139,6_23_44,0.000383
2987,6_60_42,0.000339
2043,6_41_48,0.000328


In [16]:
def get_reconstruction_analysis_df(sample_name):
    sample_index = y.tolist().index(sample_name)
    original_spectrum = X[sample_index]
    reconstructed_spectrum = all_recostruction[sample_index]
    absolute_error = np.abs(original_spectrum - reconstructed_spectrum)
    reconstruction_df = pd.DataFrame({
        'Frequency (Hz)': df.columns.values,
        'Original Spectrum': original_spectrum,
        'Reconstructed Spectrum': reconstructed_spectrum,
        'Absolute Error': absolute_error
    })
    reconstruction_df.set_index('Frequency (Hz)', inplace=True)
    # apply centered moving average to the absolute error
    reconstruction_df['Centered Moving Average absolute error'] = reconstruction_df['Absolute Error'].rolling(window=34, center=True).mean()
    return reconstruction_df

In [17]:
# take the first sample with the highest reconstruction error and plot the original spectrum and the reconstructed spectrum
sample_with_highest_error = reconstruction_error_pca_df.sort_values(by='Reconstruction Error', ascending=False).iloc[0]
reconstruction_df_highest = get_reconstruction_analysis_df(sample_with_highest_error['Sample'])
title = "Original Spectrum vs Reconstructed Spectrum for Sample: " + sample_with_highest_error['Sample']
fig = px.line(reconstruction_df_highest,
    labels={'x': 'Frequency (Hz)', 'value': 'Amplitude'},
    title=title,
    width=1200,
    height=500,
    template="plotly_white"
)
fig.update_layout(
    xaxis_title="Frequency (Hz)",
    yaxis_title="Amplitude",
)
fig.show()
sample_with_lowest_error = reconstruction_error_pca_df.sort_values(by='Reconstruction Error', ascending=True).iloc[0]
reconstruction_df_lowest = get_reconstruction_analysis_df(sample_with_lowest_error['Sample'])
title = "Original Spectrum vs Reconstructed Spectrum for Sample: " + sample_with_lowest_error['Sample']
fig = px.line(reconstruction_df_lowest,
    labels={'x': 'Frequency (Hz)', 'value': 'Amplitude'},
    title=title,
    width=1200,
    height=500,
    template="plotly_white"
)
fig.update_layout(
    xaxis_title="Frequency (Hz)",
    yaxis_title="Amplitude",
)
fig.show()

In [18]:
# all reconstruction errors
reconstruction_error_pca_all = np.concatenate((reconstruction_error_pca_train, reconstruction_error_pca_test))
df_results['reconstruction_error_pca'] = reconstruction_error_pca_all
df_results

,set,anomaly_score,reconstruction_error_pca
6_01_0,train,-0.092310,0.000040
6_01_1,train,-0.078315,0.000057
6_01_10,train,-0.086980,0.000091
6_01_11,train,-0.090074,0.000028
6_01_12,train,-0.065836,0.000042
...,...,...,...
6_60_5,test,-0.091232,0.000041
6_60_6,test,0.114529,0.000502
6_60_7,test,-0.061783,0.000560
6_60_8,test,-0.099482,0.000042


In [19]:
df_results[['anomaly_score','reconstruction_error_pca']].corr(method='spearman')

,anomaly_score,reconstruction_error_pca
anomaly_score,1.000000,0.608262
reconstruction_error_pca,0.608262,1.000000


In [20]:
# reduce dimensionality of using PACMAP
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=10, MN_ratio=0.5, FP_ratio=2.0)
X_pacmap = embedding.fit_transform(X)
df_pacmap = pd.DataFrame(X_pacmap, index=df.index, columns=[f"C{i+1}" for i in range(X_pacmap.shape[1])])
df_results["C1"] = df_pacmap["C1"]
df_results["C2"] = df_pacmap["C2"]
df_results

,set,anomaly_score,reconstruction_error_pca,C1,C2
6_01_0,train,-0.092310,0.000040,9.709760,-2.400786
6_01_1,train,-0.078315,0.000057,9.805012,-2.563612
6_01_10,train,-0.086980,0.000091,7.956083,0.981154
6_01_11,train,-0.090074,0.000028,8.908949,4.432204
6_01_12,train,-0.065836,0.000042,9.067239,-5.018116
...,...,...,...,...,...
6_60_5,test,-0.091232,0.000041,-8.566815,-10.187848
6_60_6,test,0.114529,0.000502,5.700920,-7.775596
6_60_7,test,-0.061783,0.000560,5.783148,-10.140228
6_60_8,test,-0.099482,0.000042,-8.997223,-10.089754


In [23]:
# visualize the reduced data with a 3d scatter plot using plotly
fig = px.scatter(df_results, x='C1', y='C2', title="PACMAP Reduced Data", height=800, width=1200, template="plotly_white",
                 color=df_results['anomaly_score'], size=df_results['reconstruction_error_pca'], symbol=df_results['set'],
                 hover_name=df_results.index, hover_data=['anomaly_score', 'reconstruction_error_pca', 'set'])
# move symbol legend to the top right
fig.update_layout(legend=dict(title='Set', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
# change color scale to viridis
fig.update_traces(marker=dict(colorscale='Viridis', showscale=True, colorbar=dict(title='Anomaly Score')))
fig.show()

In [24]:
# reduce dimensionality of using PACMAP (errors)
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=10, MN_ratio=0.5, FP_ratio=2.0)
X_pacmap_errors = embedding.fit_transform(df_errors.to_numpy())
df_pacmap_errors = pd.DataFrame(X_pacmap_errors, index=df.index, columns=[f"C{i+1}" for i in range(X_pacmap_errors.shape[1])])
df_results["C1_errors"] = df_pacmap_errors["C1"]
df_results["C2_errors"] = df_pacmap_errors["C2"]
df_results

,set,anomaly_score,reconstruction_error_pca,C1,C2,C1_errors,C2_errors
6_01_0,train,-0.092310,0.000040,9.709760,-2.400786,-0.846157,3.667806
6_01_1,train,-0.078315,0.000057,9.805012,-2.563612,3.263474,3.224075
6_01_10,train,-0.086980,0.000091,7.956083,0.981154,4.453114,4.110332
6_01_11,train,-0.090074,0.000028,8.908949,4.432204,-1.842226,2.345480
6_01_12,train,-0.065836,0.000042,9.067239,-5.018116,0.423294,0.501363
...,...,...,...,...,...,...,...
6_60_5,test,-0.091232,0.000041,-8.566815,-10.187848,-4.102238,4.160717
6_60_6,test,0.114529,0.000502,5.700920,-7.775596,8.628516,2.143306
6_60_7,test,-0.061783,0.000560,5.783148,-10.140228,7.134329,3.551143
6_60_8,test,-0.099482,0.000042,-8.997223,-10.089754,-0.610063,-2.108627


In [25]:
# clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10, random_state=42)
km.fit(df_results[['C1_errors', 'C2_errors']][df_results["set"] == "train"].to_numpy())
# get cluster labels for all data
clusters = km.predict(df_results[['C1_errors', 'C2_errors']].to_numpy())
# add cluster labels to results dataframe
df_results['cluster'] = clusters.astype(str)

In [26]:
df_results

,set,anomaly_score,reconstruction_error_pca,C1,C2,C1_errors,C2_errors,cluster
6_01_0,train,-0.092310,0.000040,9.709760,-2.400786,-0.846157,3.667806,5
6_01_1,train,-0.078315,0.000057,9.805012,-2.563612,3.263474,3.224075,3
6_01_10,train,-0.086980,0.000091,7.956083,0.981154,4.453114,4.110332,3
6_01_11,train,-0.090074,0.000028,8.908949,4.432204,-1.842226,2.345480,9
6_01_12,train,-0.065836,0.000042,9.067239,-5.018116,0.423294,0.501363,2
...,...,...,...,...,...,...,...,...
6_60_5,test,-0.091232,0.000041,-8.566815,-10.187848,-4.102238,4.160717,9
6_60_6,test,0.114529,0.000502,5.700920,-7.775596,8.628516,2.143306,7
6_60_7,test,-0.061783,0.000560,5.783148,-10.140228,7.134329,3.551143,7
6_60_8,test,-0.099482,0.000042,-8.997223,-10.089754,-0.610063,-2.108627,2


In [32]:
# plot clusters
# visualize the reduced data with a 3d scatter plot using plotly
fig = px.scatter(df_results, x='C1_errors', y='C2_errors', title="PACMAP on Reconstruction error Data", height=800, width=1200, template="plotly_white",
                 color=df_results['anomaly_score'], size=df_results['reconstruction_error_pca'], symbol=df_results['cluster'],
                 hover_name=df_results.index, hover_data=['anomaly_score', 'reconstruction_error_pca', 'cluster'],
                 symbol_map={str(i): str(i) for i in range(10)}, color_continuous_scale="turbo", color_continuous_midpoint=0, size_max=25, opacity=0.8)  # map cluster labels to symbols
# move symbol legend to the top right
fig.update_layout(legend=dict(title='cluster', orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1))
# change color scale to viridis
fig.update_traces(marker=dict(colorscale='Viridis', showscale=True, colorbar=dict(title='Anomaly Score')))
fig.show()

In [36]:
# take all the samples from cluster 6 and calculate the average spectrum and average reconstruction error
def get_cluster_analysis_df(cluster_label):
    cluster_samples = df_results[df_results['cluster'] == cluster_label].index
    cluster_data = df.loc[cluster_samples]
    cluster_errors = df_errors.loc[cluster_samples]
    average_spectrum = cluster_data.mean(axis=0)
    average_reconstruction_error = cluster_errors.mean(axis=0)
    
    analysis_df = pd.DataFrame({
        'Frequency (Hz)': df.columns.values,
        'Average Spectrum': average_spectrum,
        'Average Reconstruction Error': average_reconstruction_error
    })
    analysis_df.set_index('Frequency (Hz)', inplace=True)
    return analysis_df

In [37]:
# for each cluster, get the average spectrum and average reconstruction error and plot it in the same figure
clusters = df_results['cluster'].unique()
analysis_dfs = {}
for cluster in clusters:
    analysis_dfs[cluster] = get_cluster_analysis_df(cluster)
# plot the average spectrum and average reconstruction error for each cluster
fig = px.line(title='Average Spectrum and Reconstruction Error by Cluster', template='plotly_white')
for cluster, analysis_df in analysis_dfs.items():
    fig.add_scatter(x=analysis_df.index, y=analysis_df['Average Reconstruction Error'], mode='lines',
                     name='Average Reconstruction Error cluster: '+cluster, line=dict(dash='dash'))
fig.update_layout(xaxis_title='Frequency (Hz)', yaxis_title='Amplitude / Reconstruction Error')
fig.show()